In [ ]:
!pip install dialogflow gensim==3.6.0 annoy tqdm stop_words pymorphy2 python-telegram-bot==13.3

In [ ]:
!pip install pandarallel

<img src='pic_16_nlp_1.jpg' width=700>

<img src='pic_16_nlp_2.jpg' width=700>

<img src='pic_16_nlp_3.jpg' width=700>

In [ ]:
#.   RESTART KERNEL

In [24]:
import os
from telegram.ext  import Updater, CommandHandler, MessageHandler, Filters
import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import annoy
from gensim.models import Word2Vec, FastText
import pickle
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd
import pickle
    
tqdm.pandas()

from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [2]:
import mmap
import re

def get_num_lines(file_path):
    fp = open(file_path, "r+")
    buf = mmap.mmap(fp.fileno(), 0)
    lines = 0
    while buf.readline():
        lines += 1
    return lines

In [3]:
# Скачать файл Otvety.txt по ссылке
# https://drive.google.com/file/d/1DQL9ybca4USImUDaxxHmkIZNmClKBtKG/view

In [4]:
if not os.path.isfile('prepared_answers.txt'):

    question = None
    written = False

    with open("prepared_answers.txt", "w") as fout:  
        with open("Otvety.txt", "r") as fin:
            for line in tqdm_notebook(fin):
                if line.startswith("---"):
                    written = False
                    continue
                if not written and question is not None:
                    fout.write(question.replace("\t", " ").strip() + "\t" + line.replace("\t", " "))
                    written = True
                    question = None
                    continue
                if not written:
                    question = line.strip()
                    continue

In [5]:
def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [6]:
sentences = []
morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)
c = 0

file_path_from = 'Otvety.txt'
file_path_to = 'Otvety2.txt'

if not os.path.isfile(file_path_to):
    
    N = get_num_lines(file_path_from)
    with open(file_path_to, mode = 'w') as fileto:
        with open(file_path_from) as filefrom:
            for k in tqdm(range(N)):
                line = filefrom.readline()
                if line == '': break
                spls = preprocess_txt(line)
                sentences.append(spls)
                c += 1
                if c > 500000000:
                    break
                fileto.write(' '.join(spls)+'\n')
    filefrom.close()
    fileto.close()

In [7]:
sentences = []

file_path_from = 'Otvety2.txt'
if os.path.isfile(file_path_from):  
    N = get_num_lines(file_path_from) 
    with open(file_path_to, mode = 'r') as filefrom:
        for k in tqdm(range(N)):
            line = filefrom.readline()
            if line == '': break
            sentences.append(line.split())
    filefrom.close()

  0%|          | 0/7016256 [00:00<?, ?it/s]

In [14]:
vec = []
_ = [vec.extend(x)  for x in sentences[:100]]
vec = list(set(vec))
vec.sort()
vec

['1',
 '15',
 '1700',
 '2',
 '20',
 '2048х1546',
 '300вт',
 '34',
 '3g',
 '4',
 '5',
 '50',
 '500',
 '6',
 '67',
 '6последний',
 '70',
 '800',
 'acronis',
 'br',
 'brconvert',
 'brps',
 'brв',
 'brвампир',
 'brвитание',
 'brвсе',
 'brгде',
 'brдикобраз',
 'brести',
 'brзабористый',
 'brзамыкание',
 'brзеркало',
 'brзнать',
 'brи',
 'brкаждый',
 'brм',
 'brмоя',
 'brобщениеить',
 'brпереть',
 'brпо',
 'brподкормка',
 'brприветик',
 'brсорт',
 'brсуществовать',
 'brтеперь',
 'brтожий',
 'brэтота',
 'c',
 'cmd',
 'dos',
 'dove',
 'everest',
 'fat16',
 'fat32',
 'fdisk',
 'fsntfs',
 'hd',
 'ipad',
 'ipad2',
 'ipad3',
 'lte',
 'magic',
 'ntfs',
 'paragon',
 'partition',
 'speedfan',
 'susanbr',
 'x',
 'автомобиль',
 'адекватный',
 'айпад',
 'академия',
 'акронис',
 'алгоритм',
 'анализ',
 'анемия',
 'аренда',
 'бабушка',
 'бандит',
 'барьер',
 'бежать',
 'белка',
 'белый',
 'беременная',
 'беременность',
 'беречь',
 'бессознательный',
 'биополе',
 'биоэнергия',
 'благодаря',
 'благотворител

In [ ]:
#df['text'] = value_counts(normalize=True)

In [9]:
sentences[:10]

[[],
 [],
 ['вопрос', 'тдв', 'отдыхать', 'лично', 'советовать', 'завести'],
 ['хомячок'],
 ['мужик', 'йопарить', 'собачка', '50', 'кошка'],
 ['общение'],
 ['паучок'],
 ['пол', 'памытьbr', 'таг', 'тип', 'каво'],
 ['вообще', 'пообщаться'],
 ['советовать', 'сися', 'завести']]

In [9]:
#sentences = sentences[:50_000]

In [10]:
# Обучим модель FastText

file_path_from = 'ft_model'
if not os.path.isfile(file_path_from):  
    
    sentences = [i for i in tqdm(sentences) if len(i) > 2]
    modelFT = FastText(sentences=sentences, size=100, min_count=1, window=5)
    modelFT.save("ft_model")

In [48]:
modelFT = FastText.load("ft_model")
ft_index = annoy.AnnoyIndex(100 ,'angular')

In [ ]:
#Размерность вектора модели. Если установить 100 — каждое слово в корпусе будет представлено в виде 100-мерного вектора, и т.п.
#Наименьшее допустимое количество символов в слове, для которого будет создаваться векторное представление; так можно убрать частотные, но не очень значимые слова типа союзов и предлогов.
#Размер окна. Этот параметр задает, сколько соседних слов считается частью контекста. Если выставить 40, то алгоритм возьмет 40 слов спереди от слова и 40 слов сзади от слова;

In [12]:
#просто на новой версии gensim часть с фасттекстом падала, пришлось править:
#        for word in question:
#            if word in modelFT.wv:

#ну и в других местах тоже modelFT.wv
#если кто в колабе будет делать

In [15]:
set(get_stop_words("ru"))

{'а',
 'алло',
 'без',
 'близко',
 'более',
 'больше',
 'будем',
 'будет',
 'будете',
 'будешь',
 'будто',
 'буду',
 'будут',
 'будь',
 'бы',
 'бывает',
 'бывь',
 'был',
 'была',
 'были',
 'было',
 'быть',
 'в',
 'важная',
 'важное',
 'важные',
 'важный',
 'вам',
 'вами',
 'вас',
 'ваш',
 'ваша',
 'ваше',
 'ваши',
 'вверх',
 'вдали',
 'вдруг',
 'ведь',
 'везде',
 'весь',
 'вниз',
 'внизу',
 'во',
 'вокруг',
 'вон',
 'восемнадцатый',
 'восемнадцать',
 'восемь',
 'восьмой',
 'вот',
 'впрочем',
 'времени',
 'время',
 'все',
 'всегда',
 'всего',
 'всем',
 'всеми',
 'всему',
 'всех',
 'всею',
 'всю',
 'всюду',
 'вся',
 'всё',
 'второй',
 'вы',
 'г',
 'где',
 'говорил',
 'говорит',
 'год',
 'года',
 'году',
 'да',
 'давно',
 'даже',
 'далеко',
 'дальше',
 'даром',
 'два',
 'двадцатый',
 'двадцать',
 'две',
 'двенадцатый',
 'двенадцать',
 'двух',
 'девятнадцатый',
 'девятнадцать',
 'девятый',
 'девять',
 'действительно',
 'дел',
 'день',
 'десятый',
 'десять',
 'для',
 'до',
 'довольно',
 'до

In [16]:
# Создаем Индексы для вопросов-ответов

file_path_from = 'speaker.ann'
if not os.path.isfile(file_path_from):  
    morpher = MorphAnalyzer()
    sw = set(get_stop_words("ru"))
    exclude = set(string.punctuation)
    modelFT = FastText.load("ft_model")
    ft_index = annoy.AnnoyIndex(100 ,'angular')

    index_map = {}
    counter = 0
    with open("prepared_answers.txt", "r") as f:
        for line in tqdm(f):
            n_ft = 0
            spls = line.split("\t")
            index_map[counter] = spls[1]
            question = preprocess_txt(spls[0])
            vector_ft = np.zeros(100)
            for word in question:
                if word in modelFT.wv:
                    vector_ft += modelFT.wv[word]
                    n_ft += 1
            if n_ft > 0:
                vector_ft = vector_ft / n_ft
            ft_index.add_item(counter, vector_ft)

            counter += 1

            if counter > 50_000:
                break

    ft_index.build(10)
    ft_index.save('speaker.ann')
    
    with open("index_speaker.pkl", "wb") as f:
        pickle.dump(index_map, f)

In [72]:
ft_index = annoy.AnnoyIndex(100, 'angular')
ft_index.load('speaker.ann')
index_map = pd.read_pickle("index_speaker.pkl")

In [73]:
np.random.permutation(100)

array([76, 26, 39, 24, 49, 18, 95, 12, 70, 64, 86, 62, 23, 46, 11, 68, 87,
       55, 31, 34, 22, 30, 54, 28, 20, 14, 93,  7, 67, 63, 10, 29, 80, 73,
       83, 97, 98, 35,  9, 52,  2, 89, 40, 42, 77, 37,  8, 32, 57, 27, 13,
       60, 61, 66, 45,  1, 56, 51, 96, 82, 94, 21, 71, 90, 79, 85, 75, 16,
       33, 59, 81, 53, 69, 99, 44, 88,  0,  4, 41, 19, 38, 72, 58, 43,  5,
       91, 36,  3, 78, 65, 50, 47, 15, 25, 48, 74,  6, 92, 84, 17])

In [74]:
a = ft_index.get_nns_by_vector(np.random.permutation(100), 5, include_distances=True)
a

([20299, 40798, 29518, 12973, 14908],
 [1.2104798555374146,
  1.215636134147644,
  1.2166720628738403,
  1.2169945240020752,
  1.2171131372451782])

In [75]:
[index_map[x] for x in a[0]]

['от общественного транспорта чтоб его .. \n',
 'Бесплатно скачать можно здесь. <br>http://www.softportal.com/freesoftware/4741. \n',
 'жизнь надоела. \n',
 'Мне вообще ни кто не верит :))))). \n',
 'А мне всегда кого-то не хватает..... \n']

In [25]:
#. Создадим модель продуктовых данных

shop_data = pd.read_csv("ProductsDataset.csv")
#shop_data = shop_data.iloc[:5000, :]

shop_data['text'] = shop_data['title'] + " " + shop_data["descrirption"]
shop_data['text'] = shop_data['text'].progress_apply(lambda x: preprocess_txt(str(x)))
shop_data.head()

  0%|          | 0/35548 [00:00<?, ?it/s]

,title,descrirption,product_id,category_id,subcategory_id,properties,image_links,text
0,Юбка детская ORBY,"Новая, не носили ни разу. В реале красивей чем...",58e3cfe6132ca50e053f5f82,22.0,2211,"{'detskie_razmer_rost': '81-86 (1,5 года)'}",http://cache3.youla.io/files/images/360_360/58...,"[юбка, детский, orby, новый, носить, реал, кра..."
1,Ботильоны,"Новые,привезены из Чехии ,указан размер 40,но ...",5667531b2b7f8d127d838c34,9.0,902,"{'zhenskaya_odezhda_tzvet': 'Зеленый', 'visota...",http://cache3.youla.io/files/images/360_360/5b...,"[ботильон, новыепривезти, чехия, указать, разм..."
2,Брюки,Размер 40-42. Брюки почти новые - не знаю как ...,59534826aaab284cba337e06,9.0,906,{'zhenskaya_odezhda_dzhinsy_bryuki_tip': 'Брюк...,http://cache3.youla.io/files/images/360_360/59...,"[брюки, размер, 4042, брюки, новый, знать, мер..."
3,Продам детские шапки,"Продам шапки,кажда 200р.Розовая и белая проданны.",57de544096ad842e26de8027,22.0,2217,"{'detskie_pol': 'Девочкам', 'detskaya_odezhda_...",http://cache3.youla.io/files/images/360_360/57...,"[продать, детский, шапка, продать, шапкикажда,..."
4,Блузка,"Темно-синяя, 42 размер,состояние отличное,как ...",5ad4d2626c86cb168d212022,9.0,907,"{'zhenskaya_odezhda_tzvet': 'Синий', 'zhenskay...",http://cache3.youla.io/files/images/360_360/5a...,"[блузка, темносиний, 42, размерсостояние, отли..."


In [27]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

vectorizer = CountVectorizer(ngram_range=(1, 2))

In [26]:
idxs = set(np.random.randint(0, len(index_map), len(shop_data)))
# Speaker
negative_texts = [" ".join(preprocess_txt(index_map[i])) for i in tqdm(idxs)]
# Product
positive_texts = [" ".join(val) for val in tqdm(shop_data['text'].values)]

  0%|          | 0/25400 [00:00<?, ?it/s]

  0%|          | 0/35548 [00:00<?, ?it/s]

In [28]:
dataset = negative_texts + positive_texts
labels = np.zeros(len(dataset))
labels[len(negative_texts):] = np.ones(len(positive_texts))

In [29]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(dataset, labels, test_size=0.2, 
                                                    stratify=labels, random_state=13)

In [30]:
x_train_vec = vectorizer.fit_transform(X_train)
x_test_vec = vectorizer.transform(X_test)

lr = LogisticRegression().fit(x_train_vec, y_train)

In [31]:
from sklearn.metrics import accuracy_score
accuracy_score(y_true=y_test, y_pred=lr.predict(x_test_vec))

0.9761279737489745

In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer().fit(X_train)

In [33]:
np.mean(tfidf_vect.idf_)

10.77199591271185

In [34]:
idfs = {v[0]: v[1] for v in zip(tfidf_vect.vocabulary_, tfidf_vect.idf_)}

In [35]:
len(idfs.keys())

152640

In [36]:
list(idfs.keys())[:5]

['кожаный', 'накидканатуральный', 'кожа', 'отличный', 'состояние']

In [37]:
list(idfs.values())[:5]

[9.155587745156026,
 7.023960450305621,
 10.696032786103176,
 11.10149789421134,
 11.10149789421134]

In [61]:
# Создаем Индексы для продуктовых данных

file_path_from = 'shop.ann'
if not os.path.isfile(file_path_from):  
    
    
    ft_index_shop = annoy.AnnoyIndex(100 ,'angular')

    midf = np.mean(tfidf_vect.idf_)

    index_map_shop = {}
    counter = 0

    for i in tqdm(range(len(shop_data))):
        n_ft = 0
        index_map_shop[counter] = (shop_data.loc[i, "title"], shop_data.loc[i, "image_links"])
        vector_ft = np.zeros(100)
        for word in shop_data.loc[i, "text"]:
            if word in modelFT.wv:
                vector_ft += modelFT.wv[word] * idfs.get(word, midf)
                n_ft += idfs.get(word, midf)
        if n_ft > 0:
            vector_ft = vector_ft / n_ft
        ft_index_shop.add_item(counter, vector_ft)
        counter += 1

    ft_index_shop.build(10)
    ft_index_shop.save('shop.ann')

    file_path_from = 'index_shop.pkl'
    if not os.path.isfile(file_path_from):  
    
        with open("index_shop.pkl", "wb") as f:
            pickle.dump(index_map_shop, f)


In [62]:
midf = np.mean(tfidf_vect.idf_)

ft_index_shop = annoy.AnnoyIndex(100, 'angular')
ft_index_shop.load('shop.ann') 

index_map_shop = pd.read_pickle("index_shop.pkl")

In [63]:
def embed_txt(txt, idfs, midf):
    n_ft = 0
    vector_ft = np.zeros(100)
    for word in txt:
        if word in modelFT.wv:
            vector_ft += modelFT.wv[word] * idfs.get(word, midf)
            n_ft += idfs.get(word, midf)
    return vector_ft / n_ft

In [64]:
ft_index_shop.get_nns_by_vector(np.ones(100)*20, 5, include_distances=True)

([4460, 17938, 4897, 6088, 468],
 [1.20292329788208,
  1.2040966749191284,
  1.2047914266586304,
  1.205664038658142,
  1.2065114974975586])

In [ ]:
# Создание своего бота в телеграмм
# @botfather
# name1
# name1_BOT
#. ->. API

In [ ]:
# заменить на свой токен
updater = Updater("6128988575:AAFBzUvWwsZkMBt0zXVVjiAKQf4YwNFyaH0", use_context=True) # Токен API к Telegram
dispatcher = updater.dispatcher

def startCommand(update, context):
    context.bot.send_message(chat_id=update.message.chat_id, text='Привет, давай пообщаемся?')

def textMessage(update, context):
    
    input_txt = preprocess_txt(update.message.text)
    vect = vectorizer.transform([" ".join(input_txt)])
    prediction = lr.predict(vect)
    
    if prediction[0] == 1:
        vect_ft = embed_txt(input_txt, idfs, midf)
        ft_index_shop_val = ft_index_shop.get_nns_by_vector(vect_ft, 5)
        for item in ft_index_shop_val:
            title, image = index_map_shop[item]
            context.bot.send_message(chat_id=update.message.chat_id, text="title: {} image: {}".format(title, image))
        return
    vect_ft = embed_txt(input_txt, {}, 1)
    ft_index_val, distances = ft_index.get_nns_by_vector(vect_ft, 1, include_distances=True)
    if distances[0] > 2.5:
        print(distances[0])
        context.bot.send_message(chat_id=update.message.chat_id, text="Моя твоя не понимать")
        return
    context.bot.send_message(chat_id=update.message.chat_id, text=index_map[ft_index_val[0]])
        
start_command_handler = CommandHandler('start', startCommand)
text_message_handler = MessageHandler(Filters.text, textMessage)
dispatcher.add_handler(start_command_handler)
dispatcher.add_handler(text_message_handler)
updater.start_polling(clean=True)
updater.idle()

/var/folders/4s/fd42fwqn6g746zjn0s08wmxm0000gn/T/ipykernel_10879/3572922789.py:8: RuntimeWarning: invalid value encountered in divide
  return vector_ft / n_ft


In [81]:
#  https://github.com/Koziev/NLP_Datasets